# Pendulum model

<img src="../svg/pendulum_doc.svg" />



## Formulation


### Differential Algebraic Equations (DAE)

$$
\begin{split}   \nonumber
\mathbf {\dot x}  &   =  \mathbf{f (x,y,u,p) } \\
\mathbf 0 &   =  \mathbf{g (x,y,u,p) }  \\
\mathbf z &   =  \mathbf{h (x,y,u,p) } 
\end{split}
$$

with the following vectors:

* $\mathbf x$: dynamic states
* $\mathbf y$: algebraic states
* $\mathbf u$: inputs
* $\mathbf p$: parammeters
* $\mathbf f$: differential equations
* $\mathbf g$: algebraic equations	
* $\mathbf z$: outputs
* $\mathbf h$: outputs functions




$$ 
\mathbf x = \left[
\begin{array}{c} 
x_1 \\ 
x_2 \\ 
\vdots \\ 
 x_{N_x}
\end{array} \right] \;\;
\mathbf y = \left[
\begin{array}{c} 
 y_1 \\ 
 y_2 \\ 
\vdots \\ 
y_{N_y}
\end{array} \right]
$$



$$
\mathbf f = \left[
\begin{array}{c} 
 f_1 \\ 
f_2 \\ 
\vdots \\ 
f_{N_x}
\end{array} \right] \;\;
\mathbf g = \left[
\begin{array}{c} 
 g_1 \\ 
 g_2 \\ 
\vdots \\ 
 g_{N_y}
\end{array} \right]
$$

In pydae we can fomulate two systems, the first one for the so called backward solution:

$$
\begin{split} 
\mathbf {\dot x}  &  =  \mathbf {f (x,y^{ini},u^{ini},p) } \\
\mathbf 0 & =  \mathbf {g (x,y^{ini},u^{ini},p) }  
\end{split}
$$

And the second for the foreward solution:

$$
\begin{split} 
\mathbf {\dot x}  &  =  \mathbf {f (x,y^{run},u^{run},p) } \\
\mathbf 0 & =  \mathbf {g (x,y^{run},u^{run},p) }  
\end{split}
$$
    


For the particular case of the pendulum we have the following differential and algebraic equations:
$$   
\begin{eqnarray}
f_1 &=& \frac{dp_x}{dt} = v_x \\
f_2 &=& \frac{dp_y}{dt} = v_y \\
f_3 &=& \frac{dv_x}{dt} = \frac{1}{M} \left(-2 p_x \lambda + f_x - K_d v_x \right)  \\
f_4 &=& \frac{dv_y}{dt} = \frac{1}{M} \left(-M G - 2 p_y \lambda - K_d v_y \right) \\   
&&\\
g_1 &=& p_x^2 + p_y^2 - L^2  \\
g_2 &=& -\theta + \arctan\left(p_x,-p_y\right) \\
\end{eqnarray}
$$

The dynamic equations are grouped in the vector $\mathbf f$ and the dynamic states are in vector  $\mathbf x$:

$$
\mathbf{f} =
\left[
\begin{array}{c}
f_1\\
f_2\\
f_3\\
f_4
\end{array}
\right]
\;\;\;\;\;\;\;\;
\mathbf x = \left[
\begin{array}{c} 
p_x \\ 
p_y \\ 
v_x \\ 
v_y 
\end{array} \right]
$$


The considered algebraic variables and inputs for the initialization problem can be defined as follows:
$$
\mathbf {y^{ini}} = \left[
\begin{array}{c} 
 \lambda \\ 
 f_x
\end{array} \right]
\;\;\;\;\;\;\;\;
\mathbf {u^{ini}} = \left[
\begin{array}{c} 
\theta
\end{array} \right] \;\;
$$
with the pendulum angle $\theta$ as a known desired value and the unknown force $f_x$ that have to be obtained.  

Once the initialization problem is solved, the $\mathbf{u^{run}}$ vector can be obtained making $f_x$ as input, while $\theta$ is changed to an unknown algebraic state:
$$
\mathbf {y^{run}} = \left[
\begin{array}{c} 
 \lambda \\ 
\theta
\end{array} \right]
\;\;\;\;\;\;\;\;
\mathbf {u^{run}} = \left[
\begin{array}{c} 
f_x
\end{array} \right]
$$

The parameters vector for the pendulum model is as follows:
$$
\mathbf p = \left[
\begin{array}{c} 
M \\ 
G \\ 
L\\
K_d \\ 
\end{array} \right] 
$$ 

The outputs can be defined arbitrarily.
$$
\begin{split} 
\mathbf z & =  \mathbf {h (x,y^{run},u^{run},p) }  
\end{split}
$$

We may be interested in the potential energy:

$$
E_p = M G \left(p_y + L\right)
$$

And the kinetical energy:

$$
E_k = \frac{1}{2} M \left(v_x^2 + v_y^2 \right)
$$

We can now define the outputs vector as follows:
$$
\mathbf{h} =
\left[
\begin{array}{c}
M G p_y\\
\frac{1}{2} M \left(v_x^2 + v_y^2 \right)\\
f_x
\end{array}
\right]
\;\;\;\;\;\;
\mathbf{z} =
\left[
\begin{array}{c}
E_p\\
E_k\\
f_x
\end{array}
\right]
$$

## Formulation in pydae

In [2]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

### Definition of variables and constants

In [3]:
params_dict = {'L':5.21,'G':9.81,'M':10.0,'K_d':1e-3}  # parameters with default values

x_list = ['p_x','p_y','v_x','v_y']  # dynamic states

y_ini_list = ['lam','f_x']          # algebraic states for the initialization problem
u_ini_dict = {'theta':np.deg2rad(5.0)}  # input for the initialization problem

y_run_list = ['lam','theta']        # algebraic for the running problem
u_run_dict = {'f_x':0}                  # input for the running problem, its value is updated 


sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

### Equations formulation

In [4]:
# differential equations:
dp_x = v_x
dp_y = v_y
dv_x = (-2*p_x*lam + f_x - K_d*v_x)/M
dv_y = (-M*G - 2*p_y*lam - K_d*v_y)/M   

# algebraic equations
g_1 = p_x**2 + p_y**2 - L**2 -lam*1e-6
g_2 = -theta + sym.atan2(p_x,-p_y)

### Build the model

Now we will build the system. For this purpose a dictionary is required for defining the DAE structure.

In [5]:
sys = {'name':'pend1',
       'params_dict':params_dict,
       'f_list':[dp_x,dp_y,dv_x,dv_y],
       'g_list':[g_1,g_2],
       'x_list':x_list,
       'y_ini_list':y_ini_list,
       'y_run_list':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':{'E_p':M*G*(p_y+L),'E_k':0.5*M*(v_x**2+v_y**2),'f_x':f_x}}

sys = db.build(sys)

Compilation time: 4.87 s


## Model initial state

Now the model is build and we can used it to find its steady state solution or 